In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width: 100% !important; }</style>"))

# Import Dependencies

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
from scipy.optimize import curve_fit
import statsmodels.api as sm

import plotly.graph_objects as go
import plotly.express as px

In [ ]:
n = 5_000
np.random.seed(8008)
x = np.random.normal(0, 1, n)
np.random.seed(69)
z = np.random.normal(0, 1, n)
rho = 0.06
y = rho * x + np.sqrt(1 - rho**2) * z
df = pd.DataFrame({"x": x, "y": y}) / 100

In [ ]:
mod = sm.OLS(df["y"], df["x"], hasconst=False)
res = mod.fit()
print(res.summary())

In [ ]:
res.pvalues["x"]

In [ ]:
slope = res.params["x"]
r_value = res.rsquared
# Equation of the trendline
trendline_eq = f"exc ret. = {slope:.2f} * (signal val.)"
r_squared = f"R² = {r_value:.4f}"

# Generate trendline points
trendline_y = slope * df["x"]

# Simple scatter plot of some simulated data

In [ ]:
# Create the scatter plot with trendline
fig = go.Figure()

# Add scatter points
fig.add_trace(go.Scatter(
    x=df["x"],
    y=df["y"],
    mode='markers',
    marker=dict(symbol='circle-open', color='black', size=5)
))

# Add trendline
fig.add_trace(go.Scatter(
    x=df["x"],
    y=trendline_y,
    mode='lines',
    line=dict(color='blue', dash='solid')
))

# Add trendline equation and R-squared as an annotation
fig.add_annotation(
    x=1.1,  # Top right corner
    y=0.95,  # Top right corner
    text=f"{trendline_eq}<br>{r_squared}",
    showarrow=False,
    xanchor='right',
    yanchor='top',
    xref='paper',
    yref='paper',
    bgcolor='rgba(255, 255, 255, 0.8)',
    bordercolor='black',
    borderwidth=1
)

# Customize layout
fig.update_layout(
    title="Signal Value vs. Forward Excess Return, *** p << 1%",
    xaxis_title="Signal Value",
    yaxis_title="Forward Excess Return",
    template="plotly_white",
    showlegend=False,  # Remove legend
    width=800,  # Set the width of the chart
    height=600  # Set the height of the chart
)

# Show the figure
fig.show()

# Bar plot of Positions

In [ ]:
n = 200
np.random.seed(42)
x = pd.DataFrame(
    np.random.normal(0, 1, n).reshape(1, -1),
    columns=[f"asset_{i}" for i in range(n)]
)
x = x.rank(axis=1, pct=True)
x = x.sub(x.mean(axis=1), axis=0)
x = x.div(x.abs().sum(axis=1), axis=0)
x = x.T.iloc[:, 0].sort_values() * 100

fig = px.bar(x)
fig.update_yaxes(title="Portfolio Weight (%)")
fig.update_xaxes(title="Asset index", showticklabels=False)
fig.update_layout(showlegend=False, title="Positions - Ranked Function")
fig.show()


n = 200
np.random.seed(42)
x = pd.DataFrame(
    np.random.normal(0, 1, n).reshape(1, -1),
    columns=[f"asset_{i}" for i in range(n)]
)
x = x.sub(x.mean(axis=1), axis=0)
x = x.div(x.abs().sum(axis=1), axis=0)
x = x.T.iloc[:, 0].sort_values() * 100

fig = px.bar(x)
fig.update_yaxes(title="Portfolio Weight (%)")
fig.update_xaxes(title="Asset index", showticklabels=False)
fig.update_layout(showlegend=False, title="Positions - Linear Function")
fig.show()

# Cubic Profile - Positions

In [ ]:
n = 1000
xs = np.linspace(-2, 2, n)
ys = np.full(n, np.nan)
for i in range(len(xs)):
    y = xs[i] ** 7
    ys[i] = y


df = pd.DataFrame(
    {
        "x": xs,
        "y": ys,
    }
).T
min_ = df.min(axis=1)
df = (df.sub(min_, axis=0)).div(df.max(axis=1) - min_, axis=0)
df = df.T
df["y"] = df["y"] - 0.5
df["lower_err"] = df["y"] - np.random.uniform(0.1, 0.15, len(df))
df["upper_err"] = df["y"] + np.random.uniform(0.1, 0.15, len(df))
df

In [ ]:
# Create traces
fig = go.Figure()

# Main line trace
fig.add_trace(go.Scatter(
    x=df["x"],
    y=df["y"],
    mode='lines',
    name='Mean',
    line=dict(color='rgb(31, 119, 180)'),
    marker=dict(color="#444"),
    showlegend=True,
))

# Upper bound
fig.add_trace(go.Scatter(
    x=df["x"],
    y=df["upper_err"],
    name="SD",
    marker=dict(color="#444"),
    line=dict(width=0.5),
    mode='lines',
    fillcolor='rgba(68, 68, 68, 0.3)',
    showlegend=True,
))

fig.add_trace(go.Scatter(
    x=df["x"],
    y=df["lower_err"],
    name="lower",
    marker=dict(color="#444"),
    line=dict(width=0.5),
    mode='lines',
    fill='tonexty',
    fillcolor='rgba(68, 68, 68, 0.3)',
    showlegend=False
))

# Customize layout
fig.update_layout(
    title="Cubic Profile",
    xaxis_title="X",
    yaxis_title="Y",
    template="plotly_white",
    hovermode="x",
    width=1000,  # Set the width of the chart
    height=600,  # Set the height of the chart
    legend=dict(
        x=0.02,
        y=0.95,
        traceorder="reversed",
        title_font_family="Times New Roman",
        font=dict(
            family="Courier",
            size=12,
            color="black"
        ),
        bordercolor="Black",
        borderwidth=2
    )
)

fig.update_xaxes(title="Signal XS Quantile")
fig.update_yaxes(title="Forward Excess Return")
# Show the figure
fig.show()

## Fit cubic to mean line above + Generate positions

In [ ]:
def func(x, a):
    return (x + a)**3

params, _ = curve_fit(func, xdata=df["x"], ydata=df["y"])
params

In [ ]:
func(0.2, *params)

In [ ]:
n = 200
np.random.seed(42)
x = pd.DataFrame(
    np.random.normal(0, 1, n).reshape(1, -1),
    columns=[f"asset_{i}" for i in range(n)]
)
x = x.rank(axis=1, pct=True)
x = x.T.iloc[:, 0].sort_values()
x = func(x, *params)
x = x - x.mean()
x = x / x.abs().sum()
x = x * 100
x = x.sort_values()

fig = px.bar(x)
fig.update_yaxes(title="Portfolio Weight (%)")
fig.update_xaxes(title="Asset index", showticklabels=False)
fig.update_layout(showlegend=False, title="Positions - Cubic Function", height=600, width=1000)
fig.show()

## Weak short profile

In [ ]:
n = 1000
xs = np.linspace(0, 1, n)
ys = np.full(n, np.nan)
for i in range(len(xs)):
    x = xs[i]
    if x < 0.85:
        y = 0.1 * x - 0.1
    else:
        y = 2 * x - 1.715
    ys[i] = y


df = pd.DataFrame(
    {
        "x": xs,
        "y": ys,
    }
)
df["lower_err"] = df["y"] - np.random.uniform(0.05, 0.07, len(df))
df["upper_err"] = df["y"] + np.random.uniform(0.05, 0.07, len(df))


# Create traces
fig = go.Figure()

# Main line trace
fig.add_trace(go.Scatter(
    x=df["x"],
    y=df["y"],
    mode='lines',
    name='Mean',
    line=dict(color='rgb(31, 119, 180)'),
    marker=dict(color="#444"),
    showlegend=True
))

# Upper bound
fig.add_trace(go.Scatter(
    x=df["x"],
    y=df["upper_err"],
    name="SD",
    marker=dict(color="#444"),
    line=dict(width=0.5),
    mode='lines',
    fillcolor='rgba(68, 68, 68, 0.3)',
    showlegend=True,
))

fig.add_trace(go.Scatter(
    x=df["x"],
    y=df["lower_err"],
    name="lower",
    marker=dict(color="#444"),
    line=dict(width=0.5),
    mode='lines',
    fill='tonexty',
    fillcolor='rgba(68, 68, 68, 0.3)',
    showlegend=False
))

# Customize layout
fig.update_layout(
    title="Piecewise Linear",
    xaxis_title="X",
    yaxis_title="Y",
    template="plotly_white",
    hovermode="x",
    width=1000,  # Set the width of the chart
    height=600,  # Set the height of the chart
    legend=dict(
        x=0.02,
        y=0.95,
        traceorder="reversed",
        title_font_family="Times New Roman",
        font=dict(
            family="Courier",
            size=12,
            color="black"
        ),
        bordercolor="Black",
        borderwidth=2
    )
)

fig.update_xaxes(title="Signal XS Quantile")
fig.update_yaxes(title="Forward Excess Return")
# Show the figure
fig.show()

In [ ]:
n = 200
np.random.seed(42)
x = pd.DataFrame(
    np.random.normal(0, 1, n).reshape(1, -1),
    columns=[f"asset_{i}" for i in range(n)]
)
x = x.rank(axis=1, pct=True)
x = x.T.iloc[:, 0].sort_values()
x1 = x.where(x < 0.85).apply(lambda x: 0.1 * x - 0.1)
x2 = x.where(x >= 0.85).apply(lambda x: 2 * x - 1.715)
x = x1.combine_first(x2)
x = x - x.mean()
x = x / x.abs().sum()
x = x * 100
x = x.sort_values()

fig = px.bar(x)
fig.update_yaxes(title="Portfolio Weight (%)")
fig.update_xaxes(title="Asset index", showticklabels=False)
fig.update_layout(showlegend=False, title="Positions - Piecwise Linear Function",  height=600, width=1000)
fig.show()

## Non-linear, positive first decile profile

In [ ]:
n = 1000
xs = np.linspace(0, 1, n)
ys = np.full(n, np.nan)
for i in range(len(xs)):
    x = xs[i]
    if x < 0.1:
        y = -1.05 * x + 0.09
    elif 0.1 < x < 0.85:
        y = 0.02 * x - 0.02
    else:
        y = 2 * x - 1.705
    ys[i] = y


df = pd.DataFrame(
    {
        "x": xs,
        "y": ys,
    }
)
df["lower_err"] = df["y"] - np.random.uniform(0.05, 0.07, len(df))
df["upper_err"] = df["y"] + np.random.uniform(0.05, 0.07, len(df))


# Create traces
fig = go.Figure()

# Main line trace
fig.add_trace(go.Scatter(
    x=df["x"],
    y=df["y"],
    mode='lines',
    name='Mean',
    line=dict(color='rgb(31, 119, 180)'),
    marker=dict(color="#444"),
    showlegend=True,
))

# Upper bound
fig.add_trace(go.Scatter(
    x=df["x"],
    y=df["upper_err"],
    name="SD",
    marker=dict(color="#444"),
    line=dict(width=0.5),
    mode='lines',
    fillcolor='rgba(68, 68, 68, 0.3)',
    showlegend=True
))

fig.add_trace(go.Scatter(
    x=df["x"],
    y=df["lower_err"],
    name="lower",
    marker=dict(color="#444"),
    line=dict(width=0.5),
    mode='lines',
    fill='tonexty',
    fillcolor='rgba(68, 68, 68, 0.3)',
    showlegend=False
))

# Customize layout
fig.update_layout(
    title="Non-linear Profile",
    xaxis_title="X",
    yaxis_title="Y",
    template="plotly_white",
    hovermode="x",
    width=1000,  # Set the width of the chart
    height=600,  # Set the height of the chart
    legend=dict(
        x=0.02,
        y=0.95,
        traceorder="reversed",
        title_font_family="Times New Roman",
        font=dict(
            family="Courier",
            size=12,
            color="black"
        ),
        bordercolor="Black",
        borderwidth=2
    )
)

fig.update_xaxes(title="Signal XS Quantile")
fig.update_yaxes(title="Forward Excess Return")
# Show the figure
fig.show()

In [ ]:
n = 200
np.random.seed(42)
x = pd.DataFrame(
    np.random.normal(0, 1, n).reshape(1, -1),
    columns=[f"asset_{i}" for i in range(n)]
)
x = x.rank(axis=1, pct=True)
x = x.T.iloc[:, 0].sort_values()

x1 = x.where(x < 0.1).apply(lambda x: -1.05 * x + 0.09)
x2 = x.where(x >= 0.1)
x2 = x2.where(x2 < 0.85).apply(lambda x: 0.02 * x - 0.02)
x3 = x.where(x >= 0.85).apply(lambda x: 2 * x - 1.705)

x = x1.combine_first(x2)
x = x.combine_first(x3)
x = x - x.mean()
x = x / x.abs().sum()
x = x * 100
x = x.sort_values()

fig = px.bar(x)
fig.update_yaxes(title="Portfolio Weight (%)")
fig.update_xaxes(title="Asset index", showticklabels=False)
fig.update_layout(showlegend=False, title="Positions - Non-Linear Function", height=600, width=1000)
fig.show()